# Advanced SQL Operations in SQLite with Pandas

This tutorial demonstrates how to use advanced SQL operations in SQLite through Python, leveraging Pandas for data handling and display. We’ll cover `JOIN`, `GROUP BY`, `HAVING`, and `CASE` statements with separate examples for each operation.

## Setup

First, ensure you have pandas installed:

In [ ]:
!pip install pandas

Then, import the necessary libraries and set up an SQLite database with sample data.

In [2]:

import sqlite3
import pandas as pd

# Connect to an SQLite database (or create it)
conn = sqlite3.connect("example.db")

# Create sample tables: students and classes
conn.execute('''
    CREATE TABLE IF NOT EXISTS students (
        student_id INTEGER PRIMARY KEY,
        name TEXT,
        class_id INTEGER,
        age INTEGER
    )
''')

conn.execute('''
    CREATE TABLE IF NOT EXISTS classes (
        class_id INTEGER PRIMARY KEY,
        class_name TEXT
    )
''')

# Insert data into the tables
conn.executemany("INSERT OR IGNORE INTO classes (class_id, class_name) VALUES (?, ?)", 
                 [(101, 'Math'), (102, 'Science')])

conn.executemany("INSERT OR IGNORE INTO students (student_id, name, class_id, age) VALUES (?, ?, ?, ?)", 
                 [(1, 'Alice', 101, 17), (2, 'Bob', 102, 20), (3, 'Charlie', 101, 16)])

# Commit changes
conn.commit()


## 1. Using JOIN to Combine Data from Multiple Tables

The `JOIN` operation allows us to combine data from the `students` and `classes` tables. Here’s an example using `JOIN` to display each student’s name along with their class name.

In [3]:

# SQL query with JOIN
query = '''
SELECT students.name, classes.class_name
FROM students
JOIN classes ON students.class_id = classes.class_id
'''

# Use pandas to execute the query and display the results
students_classes = pd.read_sql(query, conn)
print("Students and their classes:")
print(students_classes)


Students and their classes:
      name class_name
0    Alice       Math
1      Bob    Science
2  Charlie       Math


## 2. Using GROUP BY for Aggregation

The `GROUP BY` statement allows us to aggregate data. In this example, we’ll count the number of students in each class.

In [4]:

# SQL query with GROUP BY and COUNT
query = '''
SELECT classes.class_name, COUNT(students.student_id) AS student_count
FROM students
JOIN classes ON students.class_id = classes.class_id
GROUP BY classes.class_name
'''

# Use pandas to execute the query and display the results
class_counts = pd.read_sql(query, conn)
print("\nNumber of students in each class:")
print(class_counts)



Number of students in each class:
  class_name  student_count
0       Math              2
1    Science              1


## 3. Using HAVING for Conditional Aggregation

The `HAVING` clause is used to filter groups created by `GROUP BY`. Here, we’ll filter classes to show only those with more than one student.

In [5]:

# SQL query with GROUP BY, HAVING, and COUNT
query = '''
SELECT classes.class_name, COUNT(students.student_id) AS student_count
FROM students
JOIN classes ON students.class_id = classes.class_id
GROUP BY classes.class_name
HAVING student_count > 1
'''

# Use pandas to execute the query and display the results
class_counts_filtered = pd.read_sql(query, conn)
print("\nClasses with more than one student:")
print(class_counts_filtered)



Classes with more than one student:
  class_name  student_count
0       Math              2


## 4. Using CASE for Conditional Logic

The `CASE` statement is similar to an `IF` statement in programming, allowing conditional logic in SQL queries. In this example, we’ll categorize students as 'Minor' if they’re under 18, and 'Adult' otherwise.

In [6]:

# SQL query with CASE for conditional categorization
query = '''
SELECT name, age,
    CASE 
        WHEN age < 18 THEN 'Minor'
        ELSE 'Adult'
    END AS age_category
FROM students
'''

# Use pandas to execute the query and display the results
age_categories = pd.read_sql(query, conn)
print("\nStudents categorized by age:")
print(age_categories)



Students categorized by age:
      name  age age_category
0    Alice   17        Minor
1      Bob   20        Adult
2  Charlie   16        Minor


## Closing the Database Connection

After running these queries, always close the connection to the database.

In [7]:
conn.close()

## Summary

In this tutorial, we covered the following SQL operations using SQLite and Pandas:

- `JOIN`: Combines data from multiple tables based on related columns.
- `GROUP BY`: Aggregates data by specified columns.
- `HAVING`: Filters aggregated data.
- `CASE`: Adds conditional logic within SQL queries.

Using Pandas with SQLite allows you to run SQL commands directly in Python and work with the results as DataFrames, making data manipulation and visualization easier.